# Module

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor

# Data load(우선, 15년도 데이터만)

In [61]:
df = pd.read_csv('data/back_hospital.csv')
df_back = df.drop(['Unnamed: 0'], axis=1)
df_back

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
0,20120101,강원,1,3
1,20120101,강원,2,3
2,20120101,경기,1,4
3,20120101,경기,2,5
4,20120101,경남,1,2
...,...,...,...,...
49669,20151231,제주,2,0
49670,20151231,충남,1,0
49671,20151231,충남,2,0
49672,20151231,충북,1,1


In [157]:
# df_back[df_back['back_hospital.yyyymmdd']==20150101]

In [159]:
df_back_15 = df_back.iloc[37264:]         # 15년도
df_back_15

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
37264,20150101,강원,1,4
37265,20150101,강원,2,2
37266,20150101,경기,1,4
37267,20150101,경기,2,6
37268,20150101,경남,1,6
...,...,...,...,...
49669,20151231,제주,2,0
49670,20151231,충남,1,0
49671,20151231,충남,2,0
49672,20151231,충북,1,1


In [160]:
# 한 날짜에 각 지역 별로 남,녀 17개씩 총 34개 분포
submission = pd.read_csv('data/2-2_검증데이터셋.csv', encoding='cp949')
submission

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
...,...,...,...,...
12439,2016-12-31,전남,2,NaN
12440,2016-12-31,전북,2,NaN
12441,2016-12-31,제주,2,NaN
12442,2016-12-31,충남,2,NaN


In [161]:
list(submission[submission['yyyymmdd']=='2016-01-01']['area'].values)[:17]        # 이순서대로(채울 땐, 두 겹으로.)

['강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북']

# Data Pre-processing

In [162]:
area = list(submission[submission['yyyymmdd']=='2016-01-01']['area'].values)[:17]        # 제출 순서대로
area

['강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북']

In [163]:
def preprocess(df):
    df =  pd.get_dummies(df, columns = ['back_hospital.sex'])
    df = df.reset_index()
    df = df.drop(['index', 'back_hospital.yyyymmdd', 'back_hospital.area'], axis=1)
    return df

df_back_li = []
for i in range(len(area)):
    df_back_li.append(preprocess(df_back_15[df_back_15['back_hospital.area'] == area[i]]))


df_back_li[0]

,back_hospital.frequency,back_hospital.sex_1,back_hospital.sex_2
0,4,1,0
1,2,0,1
2,1,1,0
3,1,0,1
4,1,1,0
...,...,...,...
725,1,0,1
726,2,1,0
727,0,0,1
728,2,1,0


# 강원도(index=0 file) 예시로 외부데이터 추가

In [247]:
df_back_li[0]['high'] = 0
df_back_li[0]['low'] = 0
df_back_li[0]

df0_high = pd.read_csv('data/강원_일최고기온_201501_201512.csv')    # 한 날짜당 16개 줄=> 최고는 최대, 최저는 최소로
df0_low = pd.read_csv('data/강원_일최저기온_201501_201512.csv') 

df0_low = df0_low.dropna(subset = ['hour'])
df0_high = df0_high.dropna(subset = ['hour'])

min_ = []
ind = df0_low[df0_low[df0_low.columns[0]]==' 2'].index[0]
for i in range(len(df0_low)//ind):     # 이 부분 숫자 그때마다 바뀌는 듯함  window length
    min_.append(min(df0_low[df0_low.columns[-1]].values[ind*i:ind*(i+1)-1]))
min_2 = []
for i in range(len(min_)):
    min_2.append(min_[i])
    min_2.append(min_[i])
len(min_2)


for i in range(len(df_back_li[0])-len(min_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
    min_2.append(min_2[-1])
len(min_2)

df_back_li[0]['low'] = min_2
df_back_li[0]

,back_hospital.frequency,back_hospital.sex_1,back_hospital.sex_2,high,low
0,4,1,0,0,-13.0
1,2,0,1,0,-13.0
2,1,1,0,0,-13.1
3,1,0,1,0,-13.1
4,1,1,0,0,-4.9
...,...,...,...,...,...
725,1,0,1,0,-6.6
726,2,1,0,0,-6.6
727,0,0,1,0,-6.6
728,2,1,0,0,-6.6


In [249]:
max = []
ind = df0_high[df0_high[df0_high.columns[0]]==' 2'].index[0]
for i in range(len(df0_high)//ind):
    max.append(min(df0_high[df0_high.columns[-1]].values[ind*i:ind*(i+1)-1]))
max_2 = []
for i in range(len(max)):
    max_2.append(max[i])
    max_2.append(max[i])
len(max_2)


for i in range(len(df_back_li[0])-len(max_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
    max_2.append(max_2[-1])
len(max_2)

df_back_li[0]['high'] = max_2
df_back_li[0]

,back_hospital.frequency,back_hospital.sex_1,back_hospital.sex_2,high,low
0,4,1,0,-1.9,-13.0
1,2,0,1,-1.9,-13.0
2,1,1,0,-0.9,-13.1
3,1,0,1,-0.9,-13.1
4,1,1,0,0.9,-4.9
...,...,...,...,...,...
725,1,0,1,5.1,-6.6
726,2,1,0,5.1,-6.6
727,0,0,1,5.1,-6.6
728,2,1,0,5.1,-6.6


In [ ]:
# 다른 지역 다운받고 for문 돌릴 것!

In [246]:
# df0_low[df0_low[df0_low.columns[0]]==' 2'].index[0]

16

In [201]:
# df0_low[df0_low.columns[1]].value_counts()

200.0     710
500.0     710
800.0     710
1100.0    710
1400.0    710
1700.0    710
2000.0    710
2300.0    708
Name: hour, dtype: int64

In [191]:
# len(df0_low)//16

354

In [194]:
# len(df0_low)%16

14

In [98]:
# min(df1_low[~df1_low[df1_low.columns[-1]].isna()][df1_low.columns[-1]].values)   # 최저온도=> 최소

-15.5

In [37]:
# df_back1 = df_back[df_back['back_hospital.area'] == '강원']
# df_back1_ = pd.get_dummies(df_back1, columns = ['back_hospital.sex'])

# df_back2 = df_back[df_back['back_hospital.area'] == '경기']
# df_back2_ = pd.get_dummies(df_back2, columns = ['back_hospital.sex'])

# df_back3 = df_back[df_back['back_hospital.area'] == '경남']
# df_back3_ = pd.get_dummies(df_back3, columns = ['back_hospital.sex'])

# df_back4 = df_back[df_back['back_hospital.area'] == '경북']
# df_back4_ = pd.get_dummies(df_back4, columns = ['back_hospital.sex'])

# df_back5 = df_back[df_back['back_hospital.area'] == '광주']
# df_back5_ = pd.get_dummies(df_back5, columns = ['back_hospital.sex'])

# df_back6 = df_back[df_back['back_hospital.area'] == '대구']
# df_back6_ = pd.get_dummies(df_back6, columns = ['back_hospital.sex'])

# df_back7 = df_back[df_back['back_hospital.area'] == '대전']
# df_back7_ = pd.get_dummies(df_back7, columns = ['back_hospital.sex'])

# df_back8 = df_back[df_back['back_hospital.area'] == '부산']
# df_back8_ = pd.get_dummies(df_back8, columns = ['back_hospital.sex'])

# df_back9 = df_back[df_back['back_hospital.area'] == '서울']
# df_back9_ = pd.get_dummies(df_back9, columns = ['back_hospital.sex'])

# df_back10 = df_back[df_back['back_hospital.area'] == '세종']
# df_back10_ = pd.get_dummies(df_back10, columns = ['back_hospital.sex'])

# df_back11 = df_back[df_back['back_hospital.area'] == '울산']
# df_back11_ = pd.get_dummies(df_back11, columns = ['back_hospital.sex'])

# df_back12 = df_back[df_back['back_hospital.area'] == '인천']
# df_back12_ = pd.get_dummies(df_back12, columns = ['back_hospital.sex'])

# df_back13 = df_back[df_back['back_hospital.area'] == '전남']
# df_back13_ = pd.get_dummies(df_back13, columns = ['back_hospital.sex'])

# df_back14 = df_back[df_back['back_hospital.area'] == '전북']
# df_back14_ = pd.get_dummies(df_back14, columns = ['back_hospital.sex'])

# df_back15 = df_back[df_back['back_hospital.area'] == '제주']
# df_back15_ = pd.get_dummies(df_back15, columns = ['back_hospital.sex'])

In [12]:
# len(set(submission[submission['yyyymmdd']=='2016-01-01']['area'].values))

17

In [13]:
# len(submission[submission['yyyymmdd']=='2016-01-01']['area'].values)

34

In [18]:
# 각 지역, 성별 별로 동일한 개수로 데이터 있음.
# df_back.groupby(['back_hospital.area', 'back_hospital.sex']).size().reset_index(name='counts')

,back_hospital.area,back_hospital.sex,counts
0,강원,1,1461
1,강원,2,1461
2,경기,1,1461
3,경기,2,1461
4,경남,1,1461
5,경남,2,1461
6,경북,1,1461
7,경북,2,1461
8,광주,1,1461
9,광주,2,1461


In [ ]:
# 환절기에 접어드는 10월과 1,2월은 혈관수축으로 인한 뇌혈관질환자가 특히 많이 발생합니다. 

# 그 날짜의 (단기예보)각 지역 별 최저,최고 기온의 평균-> 그 지역 기온(가장 높은/낮은 것만)으로 하고, 피쳐 생성
# 우선, 각 지역의 대표 하나(우선, 맨 위에것) 설정한 '동/읍/면/리'에서 최저,최고 이를 이용.

#----------------------------------
# 추후, 지역 뿐아니라 성별을 가를 만한 자료가 있으면 좋을 듯 => 없다면, dummy변수로.
# 시계열로 가면 좋을지
# train에서 train/val 나누고, sub을 test써서 => 데이터 수가 적어서 이를 나누어도 될지
# 계절 별로 나누어야 할까? => '기온'과 같은 피쳐를 사용한다면, 이를 대신하기 때문에 괜찮을듯함

# Train & Submission

In [ ]:
## 수정중 ##

In [ ]:
max_depth = 5
n_estimators = 5000             # 400
lr = 0.01
num_leaves = 31
early_stopping_rounds = 300      # 100
verbose = 500                  # True

In [ ]:
def model_train(curr_dir):
 
    path_train = os.path.join(curr_dir+"\\data_all\\"+"\\train")
    path_test = os.path.join(curr_dir+"\\data_all\\"+"\\test")
    
    folders_tr = os.listdir(path_train)
    folders_test = os.listdir(path_test)
    
    train = pd.DataFrame()
    for files in folders_tr:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\train\\")+files)
        train = pd.concat([train, df])            
              
    
    test = pd.DataFrame()
    for files in folders_test:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\test\\")+files)
        test = pd.concat([test, df])   
         

        
        
    X_train = train[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    X_test = test[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    y_train_LST=train[['isitu-LST']].values
    y_test_LST=test[['isitu-LST']].values
    
    y_train_TA=train[['insitu-TA']].values
    y_test_TA=test[['insitu-TA']].values
    
#     model_name = '\\save\\'+'best_model_v1.h5' 

#     filepath = os.path.join(os.getcwd()+ model_name)

#     es = EarlyStopping(monitor='val_mse', verbose=1, patience=20)

#     checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_mse',
#                              verbose=1,
#                              save_best_only=True)

#     callbacks = [checkpoint, es]

    lgbm_reg_LST = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_LST.fit(X_train, y_train_LST,
              eval_set=[(X_test, y_test_LST)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose)
    
    lgbm_reg_TA = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_TA.fit(X_train, y_train_TA,
              eval_set=[(X_test, y_test_TA)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose) 
    
 
    
#     lgbm_reg.load_weights(model_name)
    
    
    path_val = os.path.join(curr_dir+"\\data_all\\"+"\\validation")
    folders_val = os.listdir(path_val)
    
    validation = pd.DataFrame()
    for files in folders_val:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\validation\\")+files)
        validation = pd.concat([validation, df])        
        
    validation_ = validation[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    prediction_LST = lgbm_reg_LST.predict(validation_)
    prediction_TA = lgbm_reg_TA.predict(validation_)
    
    validation_partial = validation[['YearMonthDayHourMinute', 'STN']]
    validation_partial = validation_partial.assign(LST = prediction_LST)
    validation_partial = validation_partial.assign(TA = prediction_TA)
    
    submission=pd.read_csv('1-2_검증데이터셋.csv', encoding='utf-8')
    submission1=submission.copy()
    submission1[['isitu-LST', 'insitu-TA']] = validation_partial[['LST','TA']].values
    
    submission1.to_csv("lgbm_v1.csv", index = False)

In [ ]:
model_train(curr_dir)